In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import linprog
import random
import seaborn as sns

# 读取农作物信息（面积、作物类型、单价、产量等）
land_data = pd.read_excel('./附件1.xlsx',sheet_name="")
production_data = pd.read_excel('./附件2.xlsx')

# 地块面积信息
area = land_data['地块面积'].values

# 2023年农作物信息，包括单价、种植成本、产量等
crop_data = production_data[['作物类型', '产量', '单价', '种植成本', '预期销售量']].set_index('作物类型')

# 从crop_data提取变量
yield_per_acre = crop_data['产量'].values
price_per_ton = crop_data['单价'].values
cost_per_acre = crop_data['种植成本'].values

sales_expectation = crop_data['预期销售量'].values

# 变量数量
num_land_blocks = len(area)  # 地块数量
num_crops = len(crop_data)   # 作物种类数量

# 遗传算法相关参数
population_size = 50
generations = 100
mutation_rate = 0.01

# 初始化种群
def init_population(size):
    return np.random.rand(size, num_land_blocks, num_crops)

# 适应度函数，计算总利润
def fitness(individual):
    profit = 0
    for i in range(num_land_blocks):
        for j in range(num_crops):
            planted_area = individual[i, j] * area[i]
            production = planted_area * yield_per_acre[j]
            if production <= sales_expectation[j]:
                profit += production * price_per_ton[j] - planted_area * cost_per_acre[j]
            else:
                surplus = production - sales_expectation[j]
                profit += sales_expectation[j] * price_per_ton[j] + surplus * (price_per_ton[j] / 2) - planted_area * cost_per_acre[j]
    return profit

    child1.flat[point:], child2.flat[point:] = parent2.flat[point:], parent1.flat[point:]
    return child1, child2

# 变异操作
def mutate(individual):
    if np.random.rand() < mutation_rate:
        i = np.random.randint(num_land_blocks)
        j = np.random.randint(num_crops)
        individual[i, j] = np.random.rand()
    return individual

# 主遗传算法过程
def genetic_algorithm():
    population = init_population(population_size)
    best_solution = None
    best_fitness = float('-inf')
    fitness_history = []

    for generation in range(generations):
        population = selection(population)
        new_population = []

        # 交叉产生新个体
        for i in range(0, len(population), 2):
            parent1 = population[i]
            parent2 = population[min(i+1, len(population)-1)]
            child1, child2 = crossover(parent1, parent2)
            new_population.append(mutate(child1))
            new_population.append(mutate(child2))

        population = np.array(new_population)

        # 记录最佳个体
        gen_best = max(population, key=fitness)
        gen_best_fitness = fitness(gen_best)
        fitness_history.append(gen_best_fitness)

        if gen_best_fitness > best_fitness:
            best_solution = gen_best
            best_fitness = gen_best_fitness

        print(f"Generation {generation + 1}: Best Fitness = {best_fitness}")

    return best_solution, fitness_history

# 运行遗传算法
best_solution, fitness_history = genetic_algorithm()

# 总利润随代数变化趋势
plt.figure(figsize=(10, 6))
plt.plot(fitness_history, label='Total Profit')
plt.xlabel('Generations')
plt.ylabel('Profit')
plt.title('Total Profit Over Generations')
plt.legend()
plt.grid(True)
plt.show()

# 各地块的最佳作物种植方案
def plot_solution(solution):
    plt.figure(figsize=(12, 8))
    sns.heatmap(solution, annot=True, fmt=".2f", cmap='Blues', xticklabels=crop_data.index, yticklabels=land_data['地块名称'])
    plt.title("Optimal Crop Distribution Across Lands")
    plt.xlabel("Crops")
    plt.ylabel("Land Blocks")
    plt.show()

# 可视化最佳种植方案
plot_solution(best_solution)


FileNotFoundError: [Errno 2] No such file or directory: '/path/to/附件1.xlsx'

In [4]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, lpDot, value, LpStatus

# 读取预处理后的数据
data = pd.read_excel("预处理后的数据.xlsx")

# 确保数据中所有需要的列都有值
required_columns = ["作物名称", "平均价", "亩产量/斤", "种植成本/(元/亩)", "作物类型"]
if not all(col in data.columns for col in required_columns):
    raise ValueError("数据缺少必要的列")

# 提取相关数据
years = range(2024, 2031)
crops = data["作物名称"].unique()
plots = data["地块名称"].unique()

# 定义问题
prob = LpProblem("RobustProblem", LpMaximize)

# 定义变量
x = LpVariable.dicts("x", [(i, j, t) for i in plots for j in crops for t in years], lowBound=0, cat='Continuous')

# 设定目标函数
for j in crops:
    for t in years:
        # 考虑价格和产量的不确定性范围
        crop_type = data.loc[data["作物名称"] == j, "作物类型"].iloc[0]
        min_price = data.loc[data["作物名称"] == j, "平均价"].iloc[0] * (1 - 0.05) if not any(crop_type.startswith(word) for word in ["粮食"]) else data.loc[data["作物名称"] == j, "平均价"].iloc[0]
        max_price = data.loc[data["作物名称"] == j, "平均价"].iloc[0] * (1 + 0.05) if any(crop_type.startswith(word) for word in ["蔬菜"]) else data.loc[data["作物名称"] == j, "平均价"].iloc[0]
        min_yield = data.loc[data["作物名称"] == j, "亩产量/斤"].iloc[0] * (1 - 0.1)
        max_yield = data.loc[data["作物名称"] == j, "亩产量/斤"].iloc[0] * (1 + 0.1)
        min_cost = data.loc[data["作物名称"] == j, "种植成本/(元/亩)"].iloc[0] * (1 + 0.05)**(t - 2023)
        max_cost = min_cost * 1.1

        # 使用不确定参数的最坏情况进行优化
        worst_case_profit_per_unit = min_price * min_yield - max_cost
        prob += lpDot([x[i, j, t] for i in plots], worst_case_profit_per_unit)

# 设定约束条件
# 耕地资源约束
for t in years:
    prob += lpSum([x[i, j, t] for i in plots for j in crops]) <= 1201

# 种植间作的要求：同一地块同一季节不能种植两种粮食类作物
for i in plots:
    for t in years:
        grain_crops = [j for j in crops if data.loc[data["作物名称"] == j, "作物类型"].values[0].startswith("粮食")]
        prob += lpSum([x[i, j, t] for j in grain_crops]) <= 1

# 豆类作物轮作要求：每个地块三年内至少种植一次豆类作物
for i in plots:
    for t in years:
        if t <= 2026:
            prob += lpSum([x[i, j, t] for j in crops if data.loc[data["作物名称"] == j, "作物类型"].values[0].startswith("粮食（豆类）")]) >= 1

for i in plots:
    if data.loc[data["地块名称"] == i, "地块类型"].values[0] == "水浇地":
        for t in years:
            prob += lpSum([x[i, j, t] for j in crops if j == "水稻" or data.loc[data["作物名称"] == j, "作物类型"].values[0].startswith("蔬菜")]) <= 1
    else:
        for t in years:
            prob += lpSum([x[i, j, t] for j in crops if data.loc[data["作物名称"] == j, "作物类型"].values[0].startswith("粮食") and j!= "水稻"]) <= 1

# 提升非豆类作物的最低种植面积
for j in crops:
    for t in years:
        if not data.loc[data["作物名称"] == j, "作物类型"].values[0].startswith("粮食（豆类）"):
            prob += lpSum([x[i, j, t] for i in plots]) >= 30  # 设定非豆类作物最低种植面积
for j in crops:
    for t in years:
        if not data.loc[data["作物名称"] == j, "作物类型"].values[0].startswith("粮食"):
            prob += lpSum([x[i, j, t] for i in plots]) >= 20  # 设定非豆类作物最低种植面积

# 确保每个地块每年都有种植作物
for i in plots:
    for t in years:
        prob += lpSum([x[i, j, t] for j in crops]) >= 0.6  # 确保每个地块每年都至少种植0.3亩作物

# 求解问题
prob.solve()

# 检查求解器的状态
print("Status:", LpStatus[prob.status])

# 提取结果
result = pd.DataFrame([(i, j, t, value(x[i, j, t])) for i in plots for j in crops for t in years if value(x[i, j, t]) > 0], columns=["地块名称", "作物名称", "年份", "种植面积/亩"])

two_season = ["D1", "D2", "D3", "D4", "D5", "D6", "D7", "D8",
                    "E1", "E2", "E3", "E4", "E5", "E6", "E7", "E8",
                    "E9", "E10", "E11", "E12", "E13", "E14", "E15", "E16",
                    "F1", "F2", "F3", "F4"]

# 确保地块名称的列名正确
result = result.rename(columns={"地块名称": "地块名称"})

# 筛选出包含 two_season 中地块名称的行
result_two_season = result[result["地块名称"].isin(two_season)]

# 分离出第一季和第二季的数据
season1_df = result_two_season[result_two_season["地块名称"].isin(two_season) & (result_two_season["年份"] % 2 == 0)].copy()
season1_df['季节'] = "第一季"

season2_df = result_two_season[result_two_season["地块名称"].isin(two_season) & (result_two_season["年份"] % 2 == 1)].copy()
season2_df['季节'] = "第二季"

# 合并第一季和第二季的数据
result_two_season = pd.concat([season1_df, season2_df])

# 处理其他地块的第一季数据
season1_df_other = result[~result["地块名称"].isin(two_season)].copy()
season1_df_other['季节'] = "第一季"

# 合并所有数据
result = pd.concat([season1_df_other, result_two_season])
result.reset_index(drop=True, inplace=True)

# 保存结果
result.to_excel("result_robust.xlsx", index=False)

d:\Dev\Anaconda\Lib\site-packages\pulp\pulp.py:1650: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


Status: Infeasible
